In [2]:
#Import info - 
import os, re, glob, pickle
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


In [5]:
# Point this to the extracted dataset folder
DATA_DIR = Path("CleanedKARDDataset/RealWorldCoordinates")  # change if needed
assert DATA_DIR.exists(), f"Data dir not found: {DATA_DIR}"
files = sorted(glob.glob(str(DATA_DIR / "*.csv")))
print("Found files:", len(files))
print(files[:3])


Found files: 180
['CleanedKARDDataset\\RealWorldCoordinates\\a01_s01_e01_realworld.csv', 'CleanedKARDDataset\\RealWorldCoordinates\\a01_s02_e01_realworld.csv', 'CleanedKARDDataset\\RealWorldCoordinates\\a01_s03_e01_realworld.csv']


In [6]:
def parse_sequence(csv_path: str):
    """
    Parse the KARD CSV into a sequence array (T, J, 3) and the joint ordering.
    We assume a new frame whenever Joint == 'Head'.
    """
    df = pd.read_csv(csv_path)
    frames = []
    current = []
    for _, row in df.iterrows():
        if row['Joint'] == 'Head' and current:
            frames.append(pd.DataFrame(current))
            current = []
        current.append(row)
    if current:
        frames.append(pd.DataFrame(current))

    # Use the first reasonably complete frame to freeze joint order
    joint_order = None
    for fr in frames:
        if 'Joint' in fr and fr['Joint'].nunique() >= 10:
            joint_order = list(fr['Joint'])
            break
    if joint_order is None:
        return None

    T = len(frames)
    J = len(joint_order)
    seq = np.full((T, J, 3), np.nan, dtype=float)

    for t, fr in enumerate(frames):
        name_to_xyz = {n: (x, y, z) for n, x, y, z in fr[['Joint', 'x', 'y', 'z']].itertuples(index=False)}
        for j, name in enumerate(joint_order):
            if name in name_to_xyz:
                seq[t, j, :] = name_to_xyz[name]
    return seq, joint_order
